# Beira: Flood model

Including the data from floods. 

In [95]:
# load and filter osm network (step 1)

import pandas as pd
import time
import GOSTnets as gn
import networkx as nx
import osmnx as ox
import numpy as np
import geopandas as gpd
from geopandas import GeoDataFrame
import rasterio as rt
import shapely
from shapely.geometry import Point, box
from shapely.ops import unary_union
from shapely.wkt import loads
from shapely.geometry import LineString, MultiLineString, Point

# Might not use these
import fiona
import peartree
import gdal
import subprocess, glob
from PIL import Image
import numpy 
from skimage import io
from osgeo import gdal
from osgeo import osr

In [7]:
tiff_path = "../data/new_info/"

In [9]:
dem_post = io.imread(tiff_path + "Beira_DEM_Lidar_Post.tif")
dem_pre = io.imread(tiff_path + "Beira_DEM_Lidar_Pre-project.tif")


# new tifs from shapefile
pre_10yr = io.imread(tiff_path + "pre_10_year.tif")
pre_5yr = io.imread(tiff_path + "pre_5_year.tif")
post_10yr = io.imread(tiff_path + "post_10_year.tif")
post_5yr = io.imread(tiff_path + "post_5_year.tif")


In [10]:
pre_10yr.shape

(4570, 7512)

In [11]:
dem_pre.shape

(4570, 7512)

In [12]:
new_calc = dem_pre - pre_10yr

In [14]:
new_calc = dem_post - post_10yr

In [16]:
new_calc.shape

(4570, 7513)

In [17]:
np.count_nonzero(np.isnan(new_calc))


10018352

In [18]:
4570 * 7512

34329840

In [21]:
type(new_calc[0,0])

numpy.float64

In [32]:
a = dem_pre[np.where(dem_pre < -3.4028235e+37)]
a

array([-3.4028235e+38, -3.4028235e+38, -3.4028235e+38, ...,
       -3.4028235e+38, -3.4028235e+38, -3.4028235e+38], dtype=float32)

In [35]:
dem_post

array([[       nan,        nan, 6.90256964, ...,        nan,        nan,
               nan],
       [       nan,        nan, 6.90263856, ...,        nan,        nan,
               nan],
       [       nan,        nan, 6.90272402, ...,        nan,        nan,
               nan],
       ...,
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan]])

In [34]:
-3.4028235e+38

3.4028235e+38

In [42]:
dem_post = np.where(dem_post  == np.nan,-3.4028235e+38, dem_post)


In [45]:
dem_post.fillna(-3.4028235e+38)

AttributeError: 'numpy.ndarray' object has no attribute 'fillna'

In [48]:
ds = gdal.Open(tiff_path + "Beira_DEM_Lidar_Post.tif")
cols = ds.RasterXSize
rows = ds.RasterYSize
myarray = np.array(ds.GetRasterBand(1).ReadAsArray())

In [50]:
myarray.shape

(4570, 7513)

In [52]:
inDs = gdal.Open(tiff_path + "Beira_DEM_Lidar_Post.tif")


# read in the crop data and get info about it
band1 = inDs.GetRasterBand(1)
rows = inDs.RasterYSize
cols = inDs.RasterXSize

cropData = band1.ReadAsArray(0,0,cols,rows)

In [66]:
# create the output image
driver = inDs.GetDriver()
#print driver
outDs = driver.Create(tiff_path + "new_post.tif", cols, rows, 1, gdal.GDT_Int32)

In [68]:
outBand = outDs.GetRasterBand(1)
outData = numpy.zeros((rows,cols), numpy.int16)

In [76]:
outData = cropData *2

In [86]:
import numpy, sys
from osgeo import gdal
from osgeo.gdalconst import *


# register all of the GDAL drivers
gdal.AllRegister()

# open the image
inDs = gdal.Open(tiff_path + "Beira_DEM_Lidar_Post.tif")
if inDs is None:
    print('Could not open image file')
    sys.exit(1)

# read in the crop data and get info about it
band1 = inDs.GetRasterBand(1)
rows = inDs.RasterYSize
cols = inDs.RasterXSize

cropData = band1.ReadAsArray(0,0,cols,rows)


# create the output image
driver = inDs.GetDriver()
#print driver
outDs = driver.Create(tiff_path + "new_post.tif", cols, rows, bands =  1, GDT_Int32)
if outDs is None:
    print('Could not create reclass_40.tif')
    sys.exit(1)
# georeference the image and set the projection
outDs.SetGeoTransform(inDs.GetGeoTransform())
outDs.SetProjection(inDs.GetProjection())

outBand = outDs.GetRasterBand(1)
outData = numpy.zeros((rows,cols), numpy.int16)

outData = cropData*2


# write the data
outBand.WriteArray(outData)

# flush data to disk, set the NoData value and calculate stats
outBand.FlushCache()
outBand.SetNoDataValue(-99)


outDs = None
#del outData

In [132]:
import numpy 
def get_array_row_col(gdal_obj):
    band = gdal_obj.GetRasterBand(1)
    row = gdal_obj.RasterYSize
    col = gdal_obj.RasterXSize

    array = band.ReadAsArray(0,0,col,row)

    return array, row, col

tif_lst = ["Beira_DEM_Lidar_Post", "Beira_DEM_Lidar_Pre-project",
"pre_10_year", "pre_5_year", "post_10_year", "post_5_year"]

tif_files = {}
tif_arrays = {}

for tif in tif_lst:
    print(tif)
    file = gdal.Open(tiff_path + tif + ".tif")
    tif_files[tif] = file
    tif_arrays[tif] = get_array_row_col(file)
    
fin_arr = {}

fin_arr["pre_10_year"] = tif_arrays["Beira_DEM_Lidar_Pre-project"][0] - tif_arrays["pre_10_year"][0]
fin_arr["pre_5_year"]  = tif_arrays["Beira_DEM_Lidar_Pre-project"][0] - tif_arrays["pre_5_year"][0]

fin_arr["post_10_year"]  = tif_arrays["Beira_DEM_Lidar_Post"][0] - tif_arrays["post_10_year"][0]
fin_arr["post_5_year"]  =  tif_arrays["Beira_DEM_Lidar_Post"][0] - tif_arrays["post_5_year"][0]


Beira_DEM_Lidar_Post
Beira_DEM_Lidar_Pre-project
pre_10_year
pre_5_year
post_10_year
post_5_year


In [131]:
def get_driver_save_file(thing_name, cols, rows,):

    indDs = tif_files[thing_name]
    driver = indDs.GetDriver()

    if driver is None:
        print ('Could not create driver :(')

    outDs = driver.Create(tiff_path + thing_name + "_out.tif", cols, rows, bands = 1, gdal.GDT_Int32)

    if outDs is None:
        print ('Could not create outputfile :(')

    outDs.SetGeoTransform(inDs.GetGeoTransform())
    outDs.SetProjection(inDs.GetProjection())
    outBand = outDs.GetRasterBand(1)
    outBand.WriteArray(fin_arr[thing_name], 0, 0)
    outBand.FlushCache()
    outBand.SetNoDataValue(-99999)
    outDs = None


In [135]:
gdal.GDT_Int32

5

In [140]:
get_driver_save_file("pre_10_year", tif_arrays["pre_10_year"][2], tif_arrays["pre_10_year"][1])


# tif_bands['Beira_DEM_Lidar_Post'][0].ReadAsArray(0, 0, tif_bands['Beira_DEM_Lidar_Post'][1],
#                                                  tif_bands['Beira_DEM_Lidar_Post'][2])

In [141]:
tif_arrays["pre_10_year"][2]

7512

In [139]:
tif_arrays["pre_10_year"][2]

7512

In [118]:
a = tif_bands['Beira_DEM_Lidar_Post'][0]
c = a.ReadAsArray(0, 0, tif_bands['Beira_DEM_Lidar_Post'][1],
                                                 tif_bands['Beira_DEM_Lidar_Post'][2])
c

In [119]:
tif_arrays

{'Beira_DEM_Lidar_Post': None,
 'Beira_DEM_Lidar_Pre-project': None,
 'pre_10_year': None,
 'pre_5_year': None,
 'post_10_year': None,
 'post_5_year': None}

In [ ]:
    dem_post = io.imread(tiff_path + "Beira_DEM_Lidar_Post.tif")
    dem_pre = io.imread(tiff_path + "Beira_DEM_Lidar_Pre-project.tif")

    dem_post = np.nan_to_num(dem_post, nan=-3.4028235e+38)
    # new tifs from shapefile
    pre_10yr = io.imread(tiff_path + "pre_10_year")
    pre_5yr = io.imread(tiff_path + "pre_5_year")
    post_10yr = io.imread(tiff_path + "post_10_year")
    post_5yr = io.imread(tiff_path + "post_5_year")


    result_pre_10yr = dem_pre - pre_10yr
    result_pre_5yr = dem_pre - pre_5yr

    result_post_10yr = dem_post - post_10yr
    result_post_5yr = dem_post - post_5yr


    return result_pre_5yr, result_pre_10yr, result_post_5yr, result_post_10yr